In [47]:
import pickle

import os
import pandas as pd
import numpy as np
from src.utils.functions import response_real_out

eval_dir = "./results/eval/"

BENCHMARK_FILE ="./test/Glycans_q_a_v5.xlsx"

save_eval_results = "./results/eval_results_v6.xlsx"
save_maj_eval_results = "./results/eval_maj_results_v6.xlsx"
save_eval_full_results = "./results/eval_full_results_v6.xlsx"
#save_eval_overview = "./results/eval_overview.xlsx"



In [48]:
# sort=["A", "B", "C", "D"]

# sel="B"

# perm1=[3,2,0,1]

# perm2= [2,3,1,0]


# p1= [sort[el] for el in perm1] 
# print(p1)
# sel_p1=[sort[el]  for idx,el in enumerate(perm1) if sort[idx]== sel][0]
# print(sel_p1)

# p2= [sort[el] for el in perm2] 
# sel_p2=[sort[el]  for idx,el in enumerate(perm2) if sort[idx]== sel][0]
# print(sel_p2)


# rev_perm1 = [j for _,j in sorted((e,i) for i,e in enumerate(perm1))]
# print(rev_perm1)

# rev_perm2 = [j for _,j in sorted((e,i) for i,e in enumerate(perm2))]
# print(rev_perm2)


# rev_sel_p1=[sort[el] for idx, el in enumerate(rev_perm1) if sort[idx]== sel_p1][0]
# print(rev_sel_p1)

# rev_sel_p2=[sort[el] for idx, el in enumerate(rev_perm2) if sort[idx]== sel_p2][0]
# print(rev_sel_p2)


In [49]:
eval_results = [os.path.join(eval_dir, f) for f in sorted(os.listdir(eval_dir)) if f.lower().startswith('eval')]

In [50]:
el = [el for el in eval_results if el.endswith("q.pkl")][0]
print(el)
with open(el, "rb") as file:
    dt = pickle.load(file)

./results/eval/eval_Mistral_24b_4bit_mm_RAG_benchmark_20250730-111326_perm_q.pkl


In [51]:
#dt["evaluation"][3:6]

In [52]:
vd_type1 = [(el.split(".",1)[1].split("/results/eval/eval_")[1].split("_benchmark_")) for el in eval_results]

info_eval = []
for el1,el2 in vd_type1:
    if "colpali" in el1:
        tmp_type=el1.split("_")[-1:][0]
        tmp_name=el1.replace("_colpali", "")
    else:
        #print("_".join(el1.split("_")[-2:]))
        tmp_type="_".join(el1.split("_")[-2:])
        tmp_name=el1.replace("_"+tmp_type, "")
    
    if "perm_q" in el2:
        tmp_ts=el2.replace("_perm_q.pkl", "")
        tmp_perm_q = True
    else:
        tmp_ts = el2.replace(".pkl","")
        tmp_perm_q = False  
    info_eval.append({"name":tmp_name, "vd_name":tmp_type, "timestamp":tmp_ts , "perm_q":tmp_perm_q})

#print(info_eval)
#print(vd_type1)


In [53]:
#vd_type1 

In [54]:
# idx_ret= [idx for idx, el in enumerate(info_eval) if el["perm_q"]][12]


# with open(eval_results[idx_ret], "rb") as file:
#    data = pickle.load(file)

In [55]:
overview_eval =[]
eval_data = []
for el,info_el in zip(eval_results, info_eval):
   with open(el, "rb") as file:
      data = pickle.load(file)

   tmp = data["evaluation"]
   if hasattr(data, "quest_order"):
      q_order= data["quest_order"]
   else:
      q_order = [0,1,2,3]
   eval_data.append(tmp)
   data=dict((key,value) for key, value in data.items() if key != "evaluation")
   data={**data,**{"model_short":info_el["name"],"vd_name": info_el["vd_name"], "timestamp": info_el["timestamp"], "q_order": info_el["perm_q"]}}
   #data={{k:v} for k,v in data.items() if k!="evaluation"}
   overview_eval.append(data)

In [56]:
eval_dt = []
for eval, overview in zip(eval_data,overview_eval):
    for el in eval:
    #    if hasattr(el, "quest_order"):
    #     eval_dt.append({**{"model": overview["model"], "vd_name": overview["vd_name"], "timestamp": overview["timestamp"], "q_order": overview["q_order"]},  **{key: el[key] for key in ["Question_nr","answer", "quest_order"]}})
    #    else:
        eval_dt.append({**{"model": overview["model"],"model_short": overview["model_short"], "vd_name": overview["vd_name"], "timestamp": overview["timestamp"], "q_order": overview["q_order"]},  **{key: el[key] for key in ["Question_nr","answer"]}, ** {"quest_order": [0,1,2,3]}})
    

In [57]:
#els = [el for el in eval_dt if el["model"] == "google/medgemma-27b-it" and  el["vd_name"] == "no_RAG" and el["q_order"]]
#print(els)

In [58]:
for_filter = [(el["answer"], el["quest_order"]) for el in eval_dt]

In [59]:
#for_filter = [(el["answer"][1], el["quest_order"]) for el in els]

In [60]:
decode = [response_real_out(el,order)[1]  for el, order in for_filter]
 

In [61]:
# import json 
# import re

# decode = []
# error_idx=[]
# ans_list=["A", "B", "C", "D"]
# for idx, (el,q_order) in enumerate(for_filter): 
#     try:
#         if el is None:
#             decode.append("")
#         else:
#             tmp = json.loads(el)
#             match = re.search('^*(A|B|C|D)(\\s|.)', tmp.get("answer"))
#             #print(match)
#             if match and match.group(1) in ans_list:
#                 #ans_idx= [idx for idx, el in enumerate(ans_list) if match.group(1) == el][0]
#                 #real_answer = [ans_list[j] for (i,j) in enumerate(q_order) if ans_idx == i][0]
#                 #decode.append(real_answer )
#                 decode.append(match.group(1))
#             else:
#                 decode.append("")
#     except:
#         #tt=''.join(e for e in el if e.isalnum())
#         tt = ' '.join(el.split())
#         tt=tt.split(":")[-1][:10]
#         tt=tt.upper()
#         match = re.search('(A|B|C|D)(\\s|.)',tt)

#         if match and match.group(1) in ans_list:
#             #ans_idx= [idx for idx, el in enumerate(ans_list) if match.group(1) == el][0]
#             #real_answer = [ans_list[j] for (i,j) in enumerate(q_order) if ans_idx == i][0]
#             decode.append(match.group(1))
#             #decode.append(real_answer)
#         else:
#             decode.append("")

#             #print(tmp.group(1))
#             #tt=''.join(e for e in tt if e.isalnum())
#             error_idx.append(idx)
#             #print(f" idx: {idx}, value: {el[:10]}")
#         #print(tt)
#         pass


In [62]:
eval_dec =[]
for eval, answer in zip(eval_dt, decode): 
    tmp = eval
    tmp["answer"]=answer
    eval_dec.append(tmp)

#print(eval_dec)

In [63]:
#model_order= ["ibm-granite/granite-vision-3.2-2b","AdaptLLM/biomed-LLaVA-NeXT-Llama3-8B","AdaptLLM/biomed-Qwen2-VL-2B-Instruct","AdaptLLM/biomed-Llama-3.2-11B-Vision-Instruct","gpt-4o-mini","google/gemma-3-4b-it","google/gemma-3-12b-it","ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g","google/gemma-3-27b-it","Qwen/Qwen2-VL-72B-Instruct-GPTQ-Int4", "gpt-4o"]

model_order1= ["ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g","google/gemma-3-27b-it", "google/medgemma-27b-it","unsloth/Mistral-Small-3.2-24B-Instruct-2506-bnb-4bit" ,"OpenGVLab/InternVL3-14B", "gpt-4o-mini-2024-07-18","gpt-4o-2024-11-20","gpt-4.1-nano-2025-04-14","gpt-4.1-mini-2025-04-14",  "gpt-4.1-2025-04-14","Qwen/Qwen3-VL-30B-A3B-Instruct-FP8","Qwen/Qwen3-VL-30B-A3B-Thinking-FP8"]

In [64]:
eval_table = pd.DataFrame(eval_dec)

In [65]:
eval_table.model_short.unique()
model_order_short= [ 'gemma_27b','gemma_27bv2','gemma_27bv3','Mistral_24b_4bit','gemma3_internvl3-14b','medgemma-27b'
       'gemma_27b_q4', 'gpt_4.1','gpt_4o', 'gpt_4.1-mini','gpt_4o_mini', 'gpt_4.1-nano','qwen3_30b_fp8', 'qwen3_30b_think_fp8'  ]

In [66]:
#https://github.com/tompollard/tableone  
#modify this package!!

In [67]:
df_overview= pd.DataFrame(overview_eval)
df_overview.elapsed_time= df_overview.elapsed_time.round(2)
df_overview['vd_name'] = pd.Categorical(df_overview['vd_name'], ["no_RAG", "text_RAG", "mm_RAG", "colpali"])
df_overview.sort_values(by=["model","vd_name"], inplace=True)
df_overview=df_overview.reindex(columns=["model","vd_name","timestamp","q_order","elapsed_time"])

df_overview['vd_name'] = pd.Categorical(df_overview['vd_name'] , ordered=True,
                   categories=["no_RAG","text_RAG","mm_RAG","colpali"])

df_overview['model'] = pd.Categorical(df_overview['model'] , ordered=True,
                   categories=model_order1)

df_overview

,model,vd_name,timestamp,q_order,elapsed_time
65,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,20250403-222438,False,11.11
66,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,20250403-230313,False,1.61
67,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,20250403-234128,False,1.62
68,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,20250404-002024,False,1.72
69,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,20250404-005855,False,1.62
...,...,...,...,...,...
5,unsloth/Mistral-Small-3.2-24B-Instruct-2506-bn...,text_RAG,20250730-102214,True,3145.76
6,unsloth/Mistral-Small-3.2-24B-Instruct-2506-bn...,text_RAG,20250730-120618,True,3148.55
7,unsloth/Mistral-Small-3.2-24B-Instruct-2506-bn...,text_RAG,20250730-135051,True,3174.91
0,unsloth/Mistral-Small-3.2-24B-Instruct-2506-bn...,mm_RAG,20250730-111326,True,3062.17


In [68]:
df_overview1= pd.DataFrame(overview_eval)
df_overview1.elapsed_time= df_overview1.elapsed_time.round(2)
df_overview1['vd_name'] = pd.Categorical(df_overview1['vd_name'], ["no_RAG", "text_RAG", "mm_RAG", "colpali"])
df_overview1.sort_values(by=["model_short","vd_name"], inplace=True)
df_overview1=df_overview1.reindex(columns=["model_short","vd_name","timestamp","q_order","elapsed_time"])

df_overview1['vd_name'] = pd.Categorical(df_overview1['vd_name'] , ordered=True,
                   categories=["no_RAG","text_RAG","mm_RAG","colpali"])

df_overview1['model_short'] = pd.Categorical(df_overview1['model_short'] , ordered=True,
                   categories=model_order_short)
df_overview1

,model_short,vd_name,timestamp,q_order,elapsed_time
2,Mistral_24b_4bit,no_RAG,20250730-092938,True,9.76
3,Mistral_24b_4bit,no_RAG,20250730-111340,True,5.11
4,Mistral_24b_4bit,no_RAG,20250730-125746,True,4.66
5,Mistral_24b_4bit,text_RAG,20250730-102214,True,3145.76
6,Mistral_24b_4bit,text_RAG,20250730-120618,True,3148.55
...,...,...,...,...,...
209,qwen3_30b_fp8,text_RAG,20251016-115647,True,1851.70
206,qwen3_30b_fp8,mm_RAG,20251016-125103,True,3246.80
205,qwen3_30b_fp8,colpali,20251016-144210,True,6658.09
210,qwen3_30b_think_fp8,no_RAG,20251016-152228,True,2.99


In [69]:
#df_overview.to_excel(save_eval_overview)  

In [70]:
qa_data = pd.read_excel(BENCHMARK_FILE  )

In [71]:
qa_data['Difficulty'] = pd.Categorical(qa_data['Difficulty'], ordered=True,
                   categories=["Easy","Medium",	"Hard"])


#pd.pivot_table(qa_data, index = ['model','vd_name'], values = ['Cor_answer'], columns = 'Difficulty',aggfunc='mean',margins=True, margins_name='Total average')

In [72]:

pd.crosstab(index=qa_data["Difficulty"],columns="count") /pd.crosstab(index=qa_data["Difficulty"],columns="count").sum()

col_0,count
Difficulty,
Easy,0.575
Medium,0.225
Hard,0.200


In [73]:
merge_eval= eval_table.merge(qa_data, left_on='Question_nr', right_on='Question_nr')

In [74]:
merge_eval['Cor_answer'] = 1*(merge_eval['answer'] == merge_eval['Correct'])

In [75]:
merge_eval.to_excel(save_eval_full_results)  

In [76]:
#merge_eval["model"].unique()
merge_eval

,model,model_short,vd_name,timestamp,q_order,Question_nr,answer,quest_order,Paper_id,Nr_data_suppl,doi,title,question,A,B,C,D,Correct,Difficulty,Cor_answer
0,unsloth/Mistral-Small-3.2-24B-Instruct-2506-bn...,Mistral_24b_4bit,mm_RAG,20250730-111326,True,1,C,"[0, 1, 2, 3]",Paper_01,1.0,https://doi.org/10.1038/s41590-024-01916-8,A unique serum IgG glycosylation signature pre...,How many different glycan variations have been...,Over 60,Over 30,Over 40,Over 50,B,Easy,0
1,unsloth/Mistral-Small-3.2-24B-Instruct-2506-bn...,Mistral_24b_4bit,mm_RAG,20250730-111326,True,2,A,"[0, 1, 2, 3]",Paper_01,1.0,https://doi.org/10.1038/s41590-024-01916-8,A unique serum IgG glycosylation signature pre...,Which IgG Fc glycoform has been detected to be...,H3N4F1,H5N4F1,H3N4,H3N5F1,A,Easy,1
2,unsloth/Mistral-Small-3.2-24B-Instruct-2506-bn...,Mistral_24b_4bit,mm_RAG,20250730-111326,True,3,A,"[0, 1, 2, 3]",Paper_01,1.0,https://doi.org/10.1038/s41590-024-01916-8,A unique serum IgG glycosylation signature pre...,Which IgG glycan changes are seen in people wi...,decreased galactosylation,increased galactosylation,increased sialylation,decreased bisection,A,Easy,1
3,unsloth/Mistral-Small-3.2-24B-Instruct-2506-bn...,Mistral_24b_4bit,mm_RAG,20250730-111326,True,4,A,"[0, 1, 2, 3]",Paper_01,1.0,https://doi.org/10.1038/s41590-024-01916-8,A unique serum IgG glycosylation signature pre...,Which polysaccharide do ASCA antibodies bind to?,Mannan,Cellulose,Chitin,Glucan,A,Easy,1
4,unsloth/Mistral-Small-3.2-24B-Instruct-2506-bn...,Mistral_24b_4bit,mm_RAG,20250730-111326,True,5,B,"[0, 1, 2, 3]",Paper_01,1.0,https://doi.org/10.1038/s41590-024-01916-8,A unique serum IgG glycosylation signature pre...,H4N4F1 glycan structure can be described as:,Digalactosylated,Agalactosylated,Monosialylated,Monogalactosylated,D,Hard,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25435,Qwen/Qwen3-VL-30B-A3B-Thinking-FP8,qwen3_30b_think_fp8,no_RAG,20251016-175615,True,116,B,"[0, 1, 2, 3]",Paper_20,NaN,https://doi.org/10.1111/imr.13407,The importance of IgG glycosylation—What did w...,Which process is the absence of core fucose on...,increased glucose uptake,enhanced antibody-dependent cell cytotoxicity,decreased immune cell activation,impaired antigen binding,B,Easy,1
25436,Qwen/Qwen3-VL-30B-A3B-Thinking-FP8,qwen3_30b_think_fp8,no_RAG,20251016-175615,True,117,C,"[0, 1, 2, 3]",Paper_20,NaN,https://doi.org/10.1111/imr.13407,The importance of IgG glycosylation—What did w...,What is the predominant type of N-glycan struc...,High-mannose,hybrid,complex,triantennary,C,Easy,1
25437,Qwen/Qwen3-VL-30B-A3B-Thinking-FP8,qwen3_30b_think_fp8,no_RAG,20251016-175615,True,118,D,"[0, 1, 2, 3]",Paper_20,NaN,https://doi.org/10.1111/imr.13407,The importance of IgG glycosylation—What did w...,Which IgG subclass is characterized by additio...,IgG1,IgG2,IgG3,IgG4,C,Easy,0
25438,Qwen/Qwen3-VL-30B-A3B-Thinking-FP8,qwen3_30b_think_fp8,no_RAG,20251016-175615,True,119,C,"[0, 1, 2, 3]",Paper_20,NaN,https://doi.org/10.1111/imr.13407,The importance of IgG glycosylation—What did w...,Which glycan structure is described as most re...,Agalactosylated glycans,Monogalactosylated glycans,Digalactosylated glycans,Sialylated glycans,C,Medium,1


In [77]:
merge_eval['Difficulty'] = pd.Categorical(merge_eval['Difficulty'], ordered=True,
                   categories=["Easy","Medium",	"Hard"])

merge_eval['vd_name'] = pd.Categorical(merge_eval['vd_name'] , ordered=True,
                   categories=["no_RAG","text_RAG","mm_RAG","colpali"])

merge_eval['model'] = pd.Categorical(merge_eval['model'] , ordered=True,
                   categories=model_order1)
merge_eval['model_short'] = pd.Categorical(merge_eval['model_short'] , ordered=True,
                   categories=model_order_short)

merge_eval= merge_eval.merge(df_overview, on=['model','vd_name', 'timestamp','q_order'])



In [78]:
merge_eval["model"].unique()

['unsloth/Mistral-Small-3.2-24B-Instruct-2506-b..., 'google/gemma-3-27b-it', NaN, 'ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g', 'gpt-4.1-mini-2025-04-14', ..., 'gpt-4o-2024-11-20', 'gpt-4o-mini-2024-07-18', 'google/medgemma-27b-it', 'Qwen/Qwen3-VL-30B-A3B-Instruct-FP8', 'Qwen/Qwen3-VL-30B-A3B-Thinking-FP8']
Length: 12
Categories (12, object): ['ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g' < 'google/gemma-3-27b-it' < 'google/medgemma-27b-it' < 'unsloth/Mistral-Small-3.2-24B-Instruct-2506-b... ... 'gpt-4.1-mini-2025-04-14' < 'gpt-4.1-2025-04-14' < 'Qwen/Qwen3-VL-30B-A3B-Instruct-FP8' < 'Qwen/Qwen3-VL-30B-A3B-Thinking-FP8']

In [79]:
output_table = pd.pivot_table(merge_eval, index = ['model','vd_name','timestamp','q_order'], values = ['Cor_answer'], columns = 'Difficulty',aggfunc=['mean'],margins=True, margins_name='Total average', observed=True)

output_table

mean  \
                                                                        Cor_answer   
Difficulty                                                                    Easy   
model                                   vd_name timestamp       q_order              
ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g no_RAG  20250403-222438 False     0.507246   
                                                20250403-230313 False     0.478261   
                                                20250403-234128 False     0.507246   
                                                20250404-002024 False     0.492754   
                                                20250404-005855 False     0.492754   
...                                                                            ...   
Qwen/Qwen3-VL-30B-A3B-Instruct-FP8      mm_RAG  20251016-125103 True      0.695652   
                                        colpali 20251016-144210 True      0.681159   
Qwen/Qwen3-VL-30B-A3B-Thinking-FP8      no_RAG  20251016-152228 True      0.594203   
                                                20251016-175615 True      0.579710   
Total average                                                             0.676860   

                                                                                   \
                                                                                    
Difficulty                                                                 Medium   
model                                   vd_name timestamp       q_order             
ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g no_RAG  20250403-222438 False    0.444444   
                                                20250403-230313 False    0.444444   
                                                20250403-234128 False    0.444444   
                                                20250404-002024 False    0.407407   
                                                20250404-005855 False    0.444444   
...                                                                           ...   
Qwen/Qwen3-VL-30B-A3B-Instruct-FP8      mm_RAG  20251016-125103 True     0.851852   
                                        colpali 20251016-144210 True     0.777778   
Qwen/Qwen3-VL-30B-A3B-Thinking-FP8      no_RAG  20251016-152228 True     0.481481   
                                                20251016-175615 True     0.555556   
Total average                                                            0.634592   

                                                                                   \
                                                                                    
Difficulty                                                                   Hard   
model                                   vd_name timestamp       q_order             
ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g no_RAG  20250403-222438 False    0.416667   
                                                20250403-230313 False    0.416667   
                                                20250403-234128 False    0.458333   
                                                20250404-002024 False    0.416667   
                                                20250404-005855 False    0.416667   
...                                                                           ...   
Qwen/Qwen3-VL-30B-A3B-Instruct-FP8      mm_RAG  20251016-125103 True     0.500000   
                                        colpali 20251016-144210 True     0.416667   
Qwen/Qwen3-VL-30B-A3B-Thinking-FP8      no_RAG  20251016-152228 True     0.416667   
                                                20251016-175615 True     0.375000   
Total average                                                            0.519338   

                                                                                       
                                                                                       
Difficulty                                                 

In [80]:
output_table = pd.pivot_table(merge_eval, index = ['model','vd_name', 'q_order'], values = ['Cor_answer'], columns = ['Difficulty'],aggfunc=["mean"],margins=True, margins_name='Total average',observed=True)
np.round(output_table,3)

mean  \
                                                                    Cor_answer   
Difficulty                                                                Easy   
model                                              vd_name  q_order              
ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g            no_RAG   False        0.496   
                                                            True         0.420   
                                                   text_RAG False        0.719   
                                                            True         0.762   
                                                   mm_RAG   False        0.745   
                                                            True         0.765   
                                                   colpali  False        0.542   
                                                            True         0.472   
google/gemma-3-27b-it                              no_RAG   False        0.499   
                                                            True         0.427   
                                                   text_RAG False        0.786   
                                                            True         0.777   
                                                   mm_RAG   False        0.794   
                                                            True         0.776   
                                                   colpali  False        0.577   
                                                            True         0.536   
google/medgemma-27b-it                             no_RAG   True         0.464   
                                                   text_RAG True         0.768   
                                                   mm_RAG   True         0.739   
                                                   colpali  True         0.464   
unsloth/Mistral-Small-3.2-24B-Instruct-2506-bnb... no_RAG   True         0.575   
                                                   text_RAG True         0.778   
                                                   mm_RAG   True         0.732   
gpt-4o-mini-2024-07-18                             no_RAG   False        0.568   
                                                            True         0.568   
                                                   text_RAG False        0.765   
                                                            True         0.745   
                                                   mm_RAG   False        0.739   
                                                            True         0.754   
                                                   colpali  False        0.623   
                                                            True         0.609   
gpt-4o-2024-11-20                                  no_RAG   False        0.667   
                                                            True         0.664   
                                                   text_RAG False        0.823   
                                                            True         0.832   
                                                   mm_RAG   False        0.832   
                                                            True         0.835   
                                                   colpali  False        0.777   
                                                            True         0.748   
gpt-4.1-nano-2025-04-14                            no_RAG   True         0.565   
                                                   text_RAG True         0.652   
                                                   mm_RAG   True         0.652   
gpt-4.1-mini-2025-04-14                            no_RAG   True         0.797   
                                                   text_RAG True         0.826   
                                                   mm_RAG   True         0.855   
                                                   colpa

In [81]:
np.round(output_table,3).to_excel(save_eval_results)  

In [82]:
output_table1 = pd.pivot_table(merge_eval, index = ['model_short','vd_name', 'q_order'], values = ['Cor_answer'], columns = ['Difficulty'],aggfunc=["mean"],margins=True, margins_name='Total average',observed=True)
np.round(output_table1,3)

mean                            
                                      Cor_answer                            
Difficulty                                  Easy Medium   Hard Total average
model_short          vd_name  q_order                                       
gemma_27b            no_RAG   False        0.499  0.444  0.450         0.477
                              True         0.414  0.407  0.350         0.400
                     text_RAG False        0.786  0.711  0.625         0.737
                              True         0.780  0.711  0.567         0.722
                     mm_RAG   False        0.794  0.770  0.708         0.772
                              True         0.786  0.741  0.608         0.740
                     colpali  False        0.577  0.504  0.467         0.538
                              True         0.548  0.459  0.458         0.510
gemma_27bv3          no_RAG   True         0.449  0.437  0.342         0.425
                     text_RAG True         0.777  0.674  0.558         0.710
                     mm_RAG   True         0.771  0.689  0.525         0.703
Mistral_24b_4bit     no_RAG   True         0.575  0.543  0.431         0.539
                     text_RAG True         0.778  0.753  0.597         0.736
                     mm_RAG   True         0.732  0.704  0.479         0.675
gemma3_internvl3-14b no_RAG   True         0.377  0.333  0.375         0.367
                     text_RAG True         0.768  0.741  0.542         0.717
                     mm_RAG   True         0.754  0.667  0.583         0.700
                     colpali  True         0.478  0.259  0.375         0.408
gpt_4.1              no_RAG   True         0.710  0.667  0.583         0.675
                     text_RAG True         0.855  0.704  0.708         0.792
                     mm_RAG   True         0.797  0.778  0.792         0.792
                     colpali  True         0.826  0.815  0.792         0.817
gpt_4o               no_RAG   False        0.667  0.659  0.467         0.625
                              True         0.664  0.630  0.358         0.595
                     text_RAG False        0.823  0.822  0.633         0.785
                              True         0.832  0.830  0.583         0.782
                     mm_RAG   False        0.832  0.867  0.642         0.802
                              True         0.835  0.874  0.658         0.808
                     colpali  False        0.777  0.881  0.608         0.767
                              True         0.748  0.837  0.633         0.745
gpt_4.1-mini         no_RAG   True         0.797  0.667  0.417         0.692
                     text_RAG True         0.826  0.778  0.667         0.783
                     mm_RAG   True         0.855  0.778  0.667         0.800
                     colpali  True         0.855  0.815  0.583         0.792
gpt_4o_mini          no_RAG   False        0.568  0.452  0.283         0.485
                              True         0.568  0.422  0.325         0.487
                     text_RAG False        0.765  0.652  0.508         0.688
                              True         0.745  0.600  0.525         0.668
                     mm_RAG   False        0.739  0.674  0.617         0.700
                              True         0.754  0.563  0.633         0.687
                     colpali  False        0.623  0.607  0.492         0.593
                              True         0.609  0.667  0.500         0.600
gpt_4.1-nano         no_RAG   True         0.565  0.519  0.417         0.525
                     text_RAG True         0.652  0.667  0.458         0.617
                     mm_RAG   True         0.652  0.519  0.500         0.592
qwen3_30b_fp8        no_RAG   True         0.580  0.574  0.271         0.517
                     text_RAG True         0.783  0.704  0.458         0.700
                     mm_RAG   True         0.696  0.852  0.500         0.692
                     colpali  True      

In [83]:
output_table = pd.pivot_table(merge_eval, index = ['model','vd_name', 'q_order'], values = ['Cor_answer','elapsed_time'], columns = 'Difficulty',aggfunc='mean',margins=True, margins_name='Total average',observed=True)

np.round(output_table,3)

Cor_answer  \
Difficulty                                                                Easy   
model                                              vd_name  q_order              
ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g            no_RAG   False        0.496   
                                                            True         0.420   
                                                   text_RAG False        0.719   
                                                            True         0.762   
                                                   mm_RAG   False        0.745   
                                                            True         0.765   
                                                   colpali  False        0.542   
                                                            True         0.472   
google/gemma-3-27b-it                              no_RAG   False        0.499   
                                                            True         0.427   
                                                   text_RAG False        0.786   
                                                            True         0.777   
                                                   mm_RAG   False        0.794   
                                                            True         0.776   
                                                   colpali  False        0.577   
                                                            True         0.536   
google/medgemma-27b-it                             no_RAG   True         0.464   
                                                   text_RAG True         0.768   
                                                   mm_RAG   True         0.739   
                                                   colpali  True         0.464   
unsloth/Mistral-Small-3.2-24B-Instruct-2506-bnb... no_RAG   True         0.575   
                                                   text_RAG True         0.778   
                                                   mm_RAG   True         0.732   
gpt-4o-mini-2024-07-18                             no_RAG   False        0.568   
                                                            True         0.568   
                                                   text_RAG False        0.765   
                                                            True         0.745   
                                                   mm_RAG   False        0.739   
                                                            True         0.754   
                                                   colpali  False        0.623   
                                                            True         0.609   
gpt-4o-2024-11-20                                  no_RAG   False        0.667   
                                                            True         0.664   
                                                   text_RAG False        0.823   
                                                            True         0.832   
                                                   mm_RAG   False        0.832   
                                                            True         0.835   
                                                   colpali  False        0.777   
                                                            True         0.748   
gpt-4.1-nano-2025-04-14                            no_RAG   True         0.565   
                                                   text_RAG True         0.652   
                                                   mm_RAG   True         0.652   
gpt-4.1-mini-2025-04-14                            no_RAG   True         0.797   
                                                   text_RAG True         0.826   
                                                   mm_RAG   True         0.855   
                                                   colpali  True         0.855   
gpt-4.1-2025-04-14                                

In [84]:
output_table = pd.pivot_table(merge_eval, index = ['model','vd_name', 'q_order'], values = ['Cor_answer','elapsed_time'], columns = 'Difficulty',aggfunc='mean',margins=True, margins_name='Total average',observed=True)

np.round(output_table,3)

Cor_answer  \
Difficulty                                                                Easy   
model                                              vd_name  q_order              
ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g            no_RAG   False        0.496   
                                                            True         0.420   
                                                   text_RAG False        0.719   
                                                            True         0.762   
                                                   mm_RAG   False        0.745   
                                                            True         0.765   
                                                   colpali  False        0.542   
                                                            True         0.472   
google/gemma-3-27b-it                              no_RAG   False        0.499   
                                                            True         0.427   
                                                   text_RAG False        0.786   
                                                            True         0.777   
                                                   mm_RAG   False        0.794   
                                                            True         0.776   
                                                   colpali  False        0.577   
                                                            True         0.536   
google/medgemma-27b-it                             no_RAG   True         0.464   
                                                   text_RAG True         0.768   
                                                   mm_RAG   True         0.739   
                                                   colpali  True         0.464   
unsloth/Mistral-Small-3.2-24B-Instruct-2506-bnb... no_RAG   True         0.575   
                                                   text_RAG True         0.778   
                                                   mm_RAG   True         0.732   
gpt-4o-mini-2024-07-18                             no_RAG   False        0.568   
                                                            True         0.568   
                                                   text_RAG False        0.765   
                                                            True         0.745   
                                                   mm_RAG   False        0.739   
                                                            True         0.754   
                                                   colpali  False        0.623   
                                                            True         0.609   
gpt-4o-2024-11-20                                  no_RAG   False        0.667   
                                                            True         0.664   
                                                   text_RAG False        0.823   
                                                            True         0.832   
                                                   mm_RAG   False        0.832   
                                                            True         0.835   
                                                   colpali  False        0.777   
                                                            True         0.748   
gpt-4.1-nano-2025-04-14                            no_RAG   True         0.565   
                                                   text_RAG True         0.652   
                                                   mm_RAG   True         0.652   
gpt-4.1-mini-2025-04-14                            no_RAG   True         0.797   
                                                   text_RAG True         0.826   
                                                   mm_RAG   True         0.855   
                                                   colpali  True         0.855   
gpt-4.1-2025-04-14                                

In [85]:
output_table.to_excel(save_eval_results)

In [86]:
output_table1 = pd.pivot_table(merge_eval, index = ['model_short','vd_name', 'q_order'], values = ['Cor_answer','elapsed_time'], columns = 'Difficulty',aggfunc='mean',margins=True, margins_name='Total average',observed=True)

np.round(output_table1,3)

Cor_answer                              \
Difficulty                                  Easy Medium   Hard Total average   
model_short          vd_name  q_order                                          
gemma_27b            no_RAG   False        0.499  0.444  0.450         0.477   
                              True         0.414  0.407  0.350         0.400   
                     text_RAG False        0.786  0.711  0.625         0.737   
                              True         0.780  0.711  0.567         0.722   
                     mm_RAG   False        0.794  0.770  0.708         0.772   
                              True         0.786  0.741  0.608         0.740   
                     colpali  False        0.577  0.504  0.467         0.538   
                              True         0.548  0.459  0.458         0.510   
gemma_27bv3          no_RAG   True         0.449  0.437  0.342         0.425   
                     text_RAG True         0.777  0.674  0.558         0.710   
                     mm_RAG   True         0.771  0.689  0.525         0.703   
Mistral_24b_4bit     no_RAG   True         0.575  0.543  0.431         0.539   
                     text_RAG True         0.778  0.753  0.597         0.736   
                     mm_RAG   True         0.732  0.704  0.479         0.675   
gemma3_internvl3-14b no_RAG   True         0.377  0.333  0.375         0.367   
                     text_RAG True         0.768  0.741  0.542         0.717   
                     mm_RAG   True         0.754  0.667  0.583         0.700   
                     colpali  True         0.478  0.259  0.375         0.408   
gpt_4.1              no_RAG   True         0.710  0.667  0.583         0.675   
                     text_RAG True         0.855  0.704  0.708         0.792   
                     mm_RAG   True         0.797  0.778  0.792         0.792   
                     colpali  True         0.826  0.815  0.792         0.817   
gpt_4o               no_RAG   False        0.667  0.659  0.467         0.625   
                              True         0.664  0.630  0.358         0.595   
                     text_RAG False        0.823  0.822  0.633         0.785   
                              True         0.832  0.830  0.583         0.782   
                     mm_RAG   False        0.832  0.867  0.642         0.802   
                              True         0.835  0.874  0.658         0.808   
                     colpali  False        0.777  0.881  0.608         0.767   
                              True         0.748  0.837  0.633         0.745   
gpt_4.1-mini         no_RAG   True         0.797  0.667  0.417         0.692   
                     text_RAG True         0.826  0.778  0.667         0.783   
                     mm_RAG   True         0.855  0.778  0.667         0.800   
                     colpali  True         0.855  0.815  0.583         0.792   
gpt_4o_mini          no_RAG   False        0.568  0.452  0.283         0.485   
                              True         0.568  0.422  0.325         0.487   
                     text_RAG False        0.765  0.652  0.508         0.688   
                              True         0.745  0.600  0.525         0.668   
                     mm_RAG   False        0.739  0.674  0.617         0.700   
                              True         0.754  0.563  0.633         0.687   
                     colpali  False        0.623  0.607  0.492         0.593   
                              True         0.609  0.667  0.500         0.600   
gpt_4.1-nano         no_RAG   True         0.565  0.519  0.417         0.525   
                     text_RAG True         0.652  0.667  0.458         0.617   
                     mm_RAG   True         0.652  0.519  0.500         0.592   
qwen3_30b_fp8        no_RAG   True         0.580  0.574  0.271         0.517   
                     text_RAG True         0.783  0.704  0.458         0.700   
                     mm_RAG   True    

In [87]:
#pd.DataFrame(merge_eval.groupby(['model','vd_name','q_order','Difficulty','Question_nr'])['Cor_answer'].sum())

new_df = pd.pivot_table(merge_eval, index = ['model','vd_name','q_order','Question_nr','Difficulty'], values = 'Cor_answer', aggfunc=['sum','count'],observed=True).reset_index()
new_df.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in new_df.columns]
new_df

,model_,vd_name_,q_order_,Question_nr_,Difficulty_,sum_Cor_answer,count_Cor_answer
0,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,False,1,Easy,0,5
1,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,False,2,Easy,5,5
2,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,False,3,Easy,5,5
3,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,False,4,Easy,0,5
4,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,False,5,Hard,0,5
...,...,...,...,...,...,...,...
6595,Qwen/Qwen3-VL-30B-A3B-Thinking-FP8,no_RAG,True,116,Easy,2,2
6596,Qwen/Qwen3-VL-30B-A3B-Thinking-FP8,no_RAG,True,117,Easy,2,2
6597,Qwen/Qwen3-VL-30B-A3B-Thinking-FP8,no_RAG,True,118,Easy,1,2
6598,Qwen/Qwen3-VL-30B-A3B-Thinking-FP8,no_RAG,True,119,Medium,1,2


In [88]:
def get_maj_vote(nr_cor,nr_ans):
   #more than half or at least half
   return 1 if nr_cor >= -(-nr_ans//2) else 0
  
new_df["Maj_vote"]=new_df.apply(lambda x: get_maj_vote(x.sum_Cor_answer, x.count_Cor_answer), axis=1) 

In [89]:
new_df

,model_,vd_name_,q_order_,Question_nr_,Difficulty_,sum_Cor_answer,count_Cor_answer,Maj_vote
0,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,False,1,Easy,0,5,0
1,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,False,2,Easy,5,5,1
2,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,False,3,Easy,5,5,1
3,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,False,4,Easy,0,5,0
4,ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g,no_RAG,False,5,Hard,0,5,0
...,...,...,...,...,...,...,...,...
6595,Qwen/Qwen3-VL-30B-A3B-Thinking-FP8,no_RAG,True,116,Easy,2,2,1
6596,Qwen/Qwen3-VL-30B-A3B-Thinking-FP8,no_RAG,True,117,Easy,2,2,1
6597,Qwen/Qwen3-VL-30B-A3B-Thinking-FP8,no_RAG,True,118,Easy,1,2,1
6598,Qwen/Qwen3-VL-30B-A3B-Thinking-FP8,no_RAG,True,119,Medium,1,2,1


In [90]:
output_maj_vote = pd.pivot_table(new_df, index = ['model_','vd_name_', 'q_order_'], values = ['Maj_vote'], columns = 'Difficulty_',aggfunc='mean',margins=True, margins_name='Total average',observed=True)

#new_df.groupby(['model','vd_name','q_order','Difficulty'])['Maj_vote'].agg(['mean'])

In [91]:
output_maj_vote.to_excel(save_maj_eval_results)

In [92]:
np.round(output_maj_vote,3)

Maj_vote  \
Difficulty_                                                              Easy   
model_                                             vd_name_ q_order_            
ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g            no_RAG   False       0.493   
                                                            True        0.348   
                                                   text_RAG False       0.725   
                                                            True        0.783   
                                                   mm_RAG   False       0.754   
                                                            True        0.768   
                                                   colpali  False       0.551   
                                                            True        0.464   
google/gemma-3-27b-it                              no_RAG   False       0.493   
                                                            True        0.362   
                                                   text_RAG False       0.783   
                                                            True        0.783   
                                                   mm_RAG   False       0.783   
                                                            True        0.797   
                                                   colpali  False       0.594   
                                                            True        0.594   
google/medgemma-27b-it                             no_RAG   True        0.464   
                                                   text_RAG True        0.768   
                                                   mm_RAG   True        0.739   
                                                   colpali  True        0.464   
unsloth/Mistral-Small-3.2-24B-Instruct-2506-bnb... no_RAG   True        0.536   
                                                   text_RAG True        0.754   
                                                   mm_RAG   True        0.826   
gpt-4o-mini-2024-07-18                             no_RAG   False       0.536   
                                                            True        0.522   
                                                   text_RAG False       0.783   
                                                            True        0.783   
                                                   mm_RAG   False       0.739   
                                                            True        0.768   
                                                   colpali  False       0.623   
                                                            True        0.580   
gpt-4o-2024-11-20                                  no_RAG   False       0.652   
                                                            True        0.652   
                                                   text_RAG False       0.855   
                                                            True        0.826   
                                                   mm_RAG   False       0.841   
                                                            True        0.870   
                                                   colpali  False       0.783   
                                                            True        0.768   
gpt-4.1-nano-2025-04-14                            no_RAG   True        0.565   
                                                   text_RAG True        0.652   
                                                   mm_RAG   True        0.652   
gpt-4.1-mini-2025-04-14                            no_RAG   True        0.797   
                                                   text_RAG True        0.826   
                                                   mm_RAG   True        0.855   
                                                   colpali  True        0.855   
gpt-4.1-2025-04-14                                 no_RAG   True        0.710   
                   